#Import Libraries

In [ ]:
import random
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from textblob import TextBlob
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

#Methods

In [ ]:
def clean_up_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(sentence)
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens]
    return tokens

def detect_emotion(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity

    if sentiment > 0:
        return "Happy"
    elif sentiment < 0:
        return "Sad"
    else:
        return "Neutral"

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def classify(sentence):
    ERROR_THRESHOLD = 0.15
    words = pickle.load(open('words.pkl','rb'))
    classes = pickle.load(open('classes.pkl','rb'))
    model = tf.keras.models.load_model('chatbot_model.h5')
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(ints):
    tag = ints[0]['intent']
    intents = json.loads(open('intents.json').read())
    for intent in intents['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])


#Prepare Data

In [ ]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())
words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']
tf.keras.utils.set_random_seed(812)

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [ ]:
training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)
    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

#Model

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


#Training

In [ ]:

history= model.fit(trainX, trainY, epochs=100, batch_size=3, verbose=1)
model.save('chatbot_model.h5')
print('Done')

Epoch 1/100
1112/1112 [==============================] - 8s 7ms/step - loss: 5.4650 - accuracy: 0.0537
Epoch 2/100
1112/1112 [==============================] - 7s 6ms/step - loss: 5.2012 - accuracy: 0.0693
Epoch 3/100
1112/1112 [==============================] - 6s 5ms/step - loss: 5.0364 - accuracy: 0.0798
Epoch 4/100
1112/1112 [==============================] - 7s 6ms/step - loss: 4.8644 - accuracy: 0.0939
Epoch 5/100
1112/1112 [==============================] - 6s 6ms/step - loss: 4.6726 - accuracy: 0.1197
Epoch 6/100
1112/1112 [==============================] - 7s 6ms/step - loss: 4.4779 - accuracy: 0.1371
Epoch 7/100
1112/1112 [==============================] - 6s 5ms/step - loss: 4.2661 - accuracy: 0.1638
Epoch 8/100
1112/1112 [==============================] - 7s 6ms/step - loss: 4.0748 - accuracy: 0.1860
Epoch 9/100
1112/1112 [==============================] - 6s 5ms/step - loss: 3.8464 - accuracy: 0.2178
Epoch 10/100
1112/1112 [==============================] - 7s 6ms/step - l

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#Test

In [ ]:
# Load preprocessed data
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Load trained model
model = tf.keras.models.load_model('chatbot_model.h5')

while True:
    inp = input("You: ")
    if inp.lower() == "quit":
        break
    emotion = detect_emotion(inp)
    results = classify(inp+' '+emotion)
    if results:
        response = get_response(results)
        print("Bot:", response)
    else:
        print("Bot: I'm sorry, I didn't understand that.")

1/1 [==============================] - 0s 180ms/step
Bot: Stress is a natural response to challenging situations, but it's important to find healthy ways to manage it. Consider practicing relaxation techniques such as deep breathing, meditation, or yoga. Additionally, prioritize tasks, delegate responsibilities, and set boundaries to reduce stressors in your life. Remember, it's okay to ask for help and prioritize your well-being.
1/1 [==============================] - 0s 181ms/step
Bot: I'm here for you. Could you tell me why you're feeling this way?
1/1 [==============================] - 0s 69ms/step
Bot: Additionally, many workplaces offer employee assistance programs (EAPs) that provide confidential counseling services to employees and their families. Check with your HR department to see if this option is available to you.
1/1 [==============================] - 0s 76ms/step
Bot: Your body is sending you signals that it needs support. Let's discuss ways to address these physical sym